# Description
This notebook automates the task of evaluating and ranking job applications in alignment with specific job prerequisites. Initially, the program identifies numerical values from textual content and rates candidates based on their work experience. Skill evaluation is undertaken through two distinct methods. The first approach deploys a binary classification: a score of `0` is assigned if a `must-have` skill is absent from the resume, and a score of `1` is given if it's present as shown in the `Assignment4`. The second scoring mechanism involves the formulation of a criteria set which is generated in `Assignment6`, wherein skills are rated on a scale from `0` to `5` based on their alignment and relevance to the job's requirements as we see in `Assignment7`. The final skill score for each candidate is derived by multiplying the scores obtained from both methods. After extracting and processing resume data from a zip archive, the script summarizes pivotal details from each resume, including the candidate's personal information and technical competencies. Subsequently, these extracted data points assist in score computation, which integrates both skill relevance and years of experience. Concluding the process, candidates are ranked by determining the mean value of their skill and experience scores, and the consolidated details, including their rankings, are showcased in order of their job compatibility.

Now first install all the necessary libraries required to execute all functionalities within this notebook.

In [ ]:
!pip install openai
!pip install PyMuPDF
!pip install textract
!pip install python-docx
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 7.2 MB/s eta 0:00

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=45c6a986118b912ac741579a8c3626bb6aab0e21b024fb20c6fd7e364bf28676
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


Upload the .env file to the directory `/content/` which contains the "OPENAI_API_KEY"

The provided code snippet accesses sensitive values like the OpenAI API key

In [ ]:
# Export your API Key to environment variable
# Upload the .env file to the directory "/content/"
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import openai
import os
# Retrieve the API key from environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set the API key for OpenAI
openai.api_key = openai_api_key

Upload the json file containing important information about the Job requirements which was generated in Assignment1, the file containing information about the all resumes along with their summary generated from Assignment3 and the final JSON file containing the score criteria generated in Assignment6

In [ ]:
from google.colab import files

# Upload the first file
print("Please upload the first file (all_applications_summary.json):")
uploaded1 = files.upload()

# Check to ensure a file was uploaded. If not, prompt again.
while len(uploaded1) == 0:
    print("No file uploaded. Please upload the first file (all_applications_summary.json) again:")
    uploaded1 = files.upload()

# Upload the second file
print("Please upload the second file (requirements_output.json):")
uploaded2 = files.upload()

# Check to ensure a file was uploaded. If not, prompt again.
while len(uploaded2) == 0:
    print("No file uploaded. Please upload the second file (requirements_output.json) again:")
    uploaded2 = files.upload()

# Upload the third file
print("Please upload the third file (criterion_and_string_match_output.txt):")
uploaded3 = files.upload()

# Check to ensure a file was uploaded. If not, prompt again.
while len(uploaded3) == 0:
    print("No file uploaded. Please upload the third file (criterion_and_string_match_output.txt) again:")
    uploaded3 = files.upload()

# Merge the dictionaries to have all uploaded files in one
uploaded = {**uploaded1, **uploaded2, **uploaded3}

# Print details of uploaded files
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Please upload the first file (all_applications_summary.json):


Saving all_applications_summary.json to all_applications_summary.json
Please upload the second file (requirements_output.json):


Saving requirements_output.json to requirements_output.json
Please upload the third file (criterion_and_string_match_output.txt):


Saving criterion_and_string_match_output (1).txt to criterion_and_string_match_output (1).txt
User uploaded file "all_applications_summary.json" with length 34793 bytes
User uploaded file "requirements_output.json" with length 810 bytes
User uploaded file "criterion_and_string_match_output (1).txt" with length 2531 bytes


Now download the `Webinar_resumes.zip` file which contains all the resumes

In [ ]:
import requests

def download_file_from_google_drive(file_id, destination):
    base_url = "https://drive.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(base_url, params={'id': file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': file_id, 'confirm': token}
        response = session.get(base_url, params=params, stream=True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)
# Example Usage
file_id = '17V_o0Snt-Lj0FmegENPQ_rXpvWTWlZgQ'
destination = 'Webinar_resumes.zip'  # Replace with your desired file name and extension
download_file_from_google_drive(file_id, destination)

The provided code is designed to process and extract relevant details from different types of document files, specifically resumes, and subsequently summarize them. Key components include:

Importing necessary libraries such as OpenAI for interaction with the GPT model, docx for reading Word documents, textract for processing .doc files, fitz for handling PDFs, pandas for working with Excel sheets, and tiktoken for natural language processing.
1. `find_json()` searches for a JSON structure within a text.
2. `read_requirements()` and `read_json()` load data from a
specified JSON file.
3. `read_document()` reads content from various file formats including .docx, .doc, .pdf, and .xls/xlsx, converting them to plain text.
4. `check_and_trim()` trims the input text to a specified token limit using tokenization from the tiktoken library.
5. `contains_zero_or_one()` checks for the presence of either a "0" or "1" in a string, returning the respective digit if found.

In [ ]:
import openai
import json
import os
from collections import OrderedDict
import re
from docx import Document
import textract
import fitz  # PyMuPDF
import pandas as pd
import math
import tiktoken


def find_json(input_text):
    open_braces = 0
    start_idx = -1
    end_idx = -1

    for idx, char in enumerate(input_text):
        if char == "{":
            if start_idx == -1:  # Start of the outermost JSON object
                start_idx = idx
            open_braces += 1
        elif char == "}":
            open_braces -= 1
            if open_braces == 0 and start_idx != -1:  # End of the outermost JSON object
                end_idx = idx
                break  # Exiting as soon as we find the outermost closing brace

    return None if start_idx == -1 or end_idx == -1 else input_text[start_idx:end_idx+1]

def read_requirements(file_path):
    # Reads the job requirements from a JSON file
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    except Exception as e:
        print(f"Error reading requirements JSON: {e}")
        return None

def read_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def read_document(file_path):
    file_path = str(file_path)
    _, file_extension = os.path.splitext(file_path)
    text = ""
    if file_extension == '.docx':
        doc = Document(file_path)
        for para in doc.paragraphs:
            text = text + para.text + " "
    elif file_extension == '.doc':
        text = textract.process(file_path).decode()
    elif file_extension.lower() == '.pdf':
        doc = fitz.open(file_path)
        for page_number in range(len(doc)):
            page = doc[page_number]
            text = text + page.get_text() + " "
    elif file_extension.lower() in ['.xls', '.xlsx']:
        data = pd.read_excel(file_path)
        text = data.to_string(index=False)

    else:
        print(f"Unsupported file type: {file_extension}")

    return text

def check_and_trim(resume_text, max_tokens=1500):
    # tokens = nltk.word_tokenize(resume_text)
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(resume_text)
    old_len = len(tokens)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        resume_text = enc.decode(tokens)
    return resume_text, old_len, len(tokens)

def contains_zero_or_one(s):
    if re.search(r'0', s):
        return 0
    elif re.search(r'1', s):
        return 1
    return None

The code defines functions to evaluate a resume based on specific skills and criteria, leveraging OpenAI's GPT model.
1. `score_from_criterion()` takes a prompt, resume text, and generated text, and uses OpenAI's API to evaluate the resume based on the prompt. The working of it has already been discussed in Assignment7. It takes the criteria generated from Assignment6 and calculates scores (between 0 and 5) for each of the must have skills present in the resume.
2. `read_from_textfile()` reads from a predefined text file which contains the score criteria generated from Assignment6 and retrieves specific outputs related to criteria and string matching.
3. `final_score_calculation()` uses the retrieved criteria and string matches to create a prompt, which is then passed to `score_from_criterion()` to generate a score for the resume based on the required skills.
4. `initial_score()` is another function that, given a list of "must-have" skills, evaluates the resume to find projects where these skills were applied, summarizing the project and assigning a score `(0 or 1)` based on the presence or absence of the skill in the project. The results from both scoring methods are represented in JSON format. This binary scoring system has already been discussed in Assignment4


In [ ]:
def score_from_criterion(prompt, text, generated_text):
    model="gpt-3.5-turbo-16k"
    max_tokens=2000
    # print("prompt", prompt)
    messages = [
            {"role": "assistant", "content": f"{generated_text}"},
            {"role": "system", "content": f"{prompt}"},
            {"role": "user", "content": f"{text}"},
        ]
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=0,
            max_tokens=max_tokens
        )
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    return generated_texts[0]

def read_from_textfile(filename):
    with open(filename, 'r') as file:
        content = file.read()

    # Separate criterion_gen_output and string_match
    criterion_start = content.find("----Criterion Generated Output----") + len("----Criterion Generated Output----")
    criterion_end = content.find("----String Match----")

    criterion_gen_output = content[criterion_start:criterion_end].strip()
    string_match = content[criterion_end + len("----String Match----"):].strip()

    return criterion_gen_output, string_match


def final_score_calculation(resume_text, must_have_skills):
    # Read the criteria and string_match from the text file
    criterion_gen_output, string_match = read_from_textfile("/content/criterion_and_string_match_output.txt")
    prompt=f'''You are an assistant to a recruiter. \
    Use the criteria given by {criterion_gen_output} to judge the resume given to you. \
    The scores must be given with respect to each of the must have skills present here {must_have_skills}, which \
    can be found inside the resume. \
    Return the output in JSON format.'''

    # Assuming the function score_from_criterion exists and takes these parameters
    score_output = score_from_criterion(prompt, resume_text, criterion_gen_output)
    # print("final scoreeee final_score_func", score_output)
    return score_output

def initial_score(text, must_have_skills):
    model="gpt-3.5-turbo-16k"
    max_tokens=2000
    first_prompt = f'''You are an assistant to a recruiter. Your job is to evaluate a resume for a particular skill.  The skills for which you need to do
    evaluation are {must_have_skills}. You need to find the projects in which a particular skill from this list has been applied. For each skill, \
    use the technical skill as the key and this key will further have 2 more keys "summary" and "score". "score" is 0 if there is no project using this \
    particular skill and "summary" is empty and "score" is 1 if you find a project with the particular technical skill, then use "summary" to \
    explain the project. Now do this for all the must have skills. \
    Return your response as a JSON'''
    messages = [
            {"role": "system", "content": f"{first_prompt}"},
            {"role": "user", "content": text },
        ]
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            temperature=1,
            max_tokens=max_tokens
        )
    generated_texts = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    # print("initial score calculation", generated_texts)
    return generated_texts[0]

The function `calculate_score` assesses a candidate's resume against a set of "must-have" skills by leveraging two different evaluation functions (`initial_score` and `final_score_calculation`). The results of both scoring methods are then combined, producing a cumulative score for each skill, which is determined by multiplying the scores from both evaluations. It also accumulates justifications and summaries. The function then computes an overall score by considering the number of skills with non-zero scores relative to the total number of skills evaluated, providing a normalized representation of a candidate's qualifications. The final output includes the overall score, count of evaluated skills, individual skill scores, and the combined scores with associated justifications and summaries.

In [ ]:
import re

def calculate_score(technical_skills, text_sum, mskill, max_retries=1):
    def process_request(text_sum, mskill, match_func, max_retries):
        retries = 0
        req_dict = {}

        while retries < max_retries:
            req_json = match_func(text_sum, mskill)
            # print("ssssssssss", req_json, match_func)
            req_json = find_json(req_json)
            # print("s11111", req_json)

            if isinstance(req_json, dict):
                req_dict = req_json
                if req_dict:  # If the dictionary is not empty, return it
                    return req_dict

            elif isinstance(req_json, str):
                try:
                    req_dict = json.loads(req_json)
                    if req_dict:  # If the dictionary is not empty, return it
                        return req_dict
                except json.JSONDecodeError:
                    try:
                        dict_from_str_repr = eval(req_json)
                        if isinstance(dict_from_str_repr, dict):
                            req_dict = dict_from_str_repr
                            if req_dict:  # If the dictionary is not empty, return it
                                return req_dict
                    except:
                        pass

            retries += 1
        return req_dict  # Returning the processed req_dict if not returned earlier

    req_dict_v1 = process_request(text_sum, mskill, initial_score, max_retries)
    req_dict_v2 = process_request(text_sum, mskill, final_score_calculation, max_retries)

    # Combine scores from both dictionaries and their justifications/summaries
    combined_scores = {}
    for skill in set(list(req_dict_v1.keys()) + list(req_dict_v2.keys())):  # ensuring we get all skills from both dicts
        score_v1 = req_dict_v1.get(skill, {}).get('score', 0)
        score_v2 = req_dict_v2.get(skill, {}).get('score', 0)
        summary_v1 = req_dict_v1.get(skill, {}).get('summary', '')
        justification_v2 = req_dict_v2.get(skill, {}).get('justification', '')
        final_score = score_v1 * score_v2
        combined_scores[skill] = {
            'combined_score': final_score,
            'summary': summary_v1,
            'justification': justification_v2
        }

    non_zero_skills = 0  # Count of skills with non-zero combined scores
    total_score = 0

    store_js = {}
    for skill, data in combined_scores.items():
        score = data.get('combined_score', 0)
        store_js[skill] = score
        if score > 0:
            non_zero_skills += 1
        total_score += score

    count = len(combined_scores)

    factor = 1 + non_zero_skills
    if count == 0:
        overall_score = 0
    else:
        overall_score = (factor / (count + 1)) * (total_score / count)

    return overall_score, count + 1, store_js, combined_scores


The provided code includes several utility functions to aid in evaluating and ranking job applications based on the experience of candidates:

**get_numerical_val(s):**
This function takes a string s as input and returns the first numerical value found in it. If the string represents an integer, it's returned as an int; if it's a floating point, it's returned as a float. If no numerical value is found, it returns None.

**score_experience(candidate_experience, job_experience_range):**
Accepting a candidate's experience (candidate_experience) and a desired job experience range (job_experience_range), it computes a score for the candidate based on how closely their experience matches the desired range. If the candidate's experience exceeds the maximum expected experience, a decay factor is applied to lower their score.

**rank_applications(applications):**
This function sorts the list of applications based on a mean score, which is the average of the general score (score) and the skills score (skill_score) of each application, normalized by the number of must-have skills. Applications are ranked in descending order of this mean score, and a rank is assigned to each application.

**extract_years(text):**
Given a text string, this function aims to extract year values. If it finds only one year value, it returns that year and a subsequent year (the given year + 5). If two year values are detected, it returns them both as a range. If no valid year range can be extracted, it returns None.

**years_of_exp(application, range_list, skill_score):**
This function updates the provided application dictionary with scores calculated based on the years of experience. Using the get_numerical_val function, it retrieves the years of experience from the application and calculates a score using the score_experience function. It updates the score and skill_score of the application based on this.

These utilities collectively aim to rank candidates based on the importance of their skills and the match between their years of experience and the job's requirements.

In [ ]:
import zipfile
import shutil
def get_numerical_val(s):
    # number = re.findall(r'[+-]?([0-9]*[.])?[0-9]+', s)
    s = str(s)
    number = re.findall(r'([0-9]*[.])?[0-9]+', s)
    if s.isdigit():
        return int(s)
    if number:
        # If there is a decimal point in the number,
        # return it as a float; otherwise, return as int.
        if '.' in number[0]:
            return float(number[0])
        else:
            number = re.findall(r'[0-9]+', s)
            return int(number[0])
    else:
        return None

def score_experience(candidate_experience, job_experience_range):
    min_exp, max_exp = job_experience_range
    alpha = 1 / (max_exp - min_exp)**2
    decay_multiplier = 0.5

    if candidate_experience > max_exp:
        alpha *= decay_multiplier
    score = math.exp(-alpha * (candidate_experience - max_exp)**2)

    return score
def rank_applications(applications):
    # Calculate the mean for each application
    for application in applications:
        if application["score"] is None or application["skill_score"] is None:
            application["mean"] = -1
        else:
            application["mean"] = (float(application["score"]) + float(application["skill_score"])) / application["len_must_have"]

    # Sort applications based on the mean
    applications.sort(key=lambda x: x["mean"], reverse=True)

    # Rank the applications
    for rank, app in enumerate(applications):
        app["rank"] = rank + 1  # Update rank

    return applications

def extract_years(text):
    numbers = re.findall(r'(\d+)', text)
    if len(numbers) == 1:
        return [int(numbers[0]), int(numbers[0])+5]
    elif len(numbers) >= 2:
        return [int(numbers[0]), int(numbers[1])]
    return None

def years_of_exp(application, range_list, skill_score):
    output = 0
    print(get_numerical_val(application["years_of_experience"]), "ssssss")
    yoe = int(get_numerical_val(application["years_of_experience"]))
    if len(range_list) == 1:
        val = range_list.pop()

        range_list.append(int(val)-1)
        range_list.append(int(val))
    output = score_experience(yoe, range_list)
    application["score"] = output
    application["skill_score"] = skill_score
    return application

The code provided aims to rank job applications based on candidates' experience and technical skills, in accordance with a predefined set of job requirements. Here's a step-by-step breakdown:

**Loading Job Requirements:**
The code starts by reading the job requirements from JSON file generated in Assignment1. The job requirements include a list of essential skills (must_have_skills) and the desired years of experience (yoe).

**Loading Candidate Data: **
The code then loads the data of filtered applications from JSON file generated in Assignment3

**Extract and Rename Resumes: **
Using the extract_and_rename function, the resumes of applicants (zipped in Webinar_resumes.zip) are extracted to a directory. This function also renames directories that have spaces in their names, replacing spaces with underscores.

**Processing Applications:**
In the **process_applications** function, each candidate's application is processed:

1. For each application, it reads the resume text and trims it if needed.
2. It then extracts the candidate's name and years of experience from the summarized resume data.
3. The candidate's technical skills are evaluated against the must_have_skills using the calculate_score function (not provided in the shared code). This function apparently returns a score based on the match between the candidate's skills and the job requirements.
4. Additionally, the desired years of experience is processed, and a score is assigned to the candidate based on their years of experience.
5. Finally, relevant data points such as the name, score, and skills of the candidate are saved to the application.
6. Ranking the Applications: The rank_applications function (explained in a previous breakdown) is then called to rank the applications based on their mean score, which is the average of their general score and skills score normalized by the number of must-have skills.

**Displaying the Rankings:**
After ranking, the code displays the ranking details for each candidate, showing:

1. The candidate's name
2. Years of experience
3. Skill score
4. List of skills
5. The final score (mean score)

In essence, this code provides an automated system to score and rank job applications based on how well the candidate's skills and experience align with the job's requirements.

In [ ]:
job_requirements = read_requirements('/content/requirements_output.json')
must_have_skills = job_requirements["must_have_skills"]
json_data = read_json('/content/all_applications_summary.json')
yoe = job_requirements["year_of_experience"]

# Process and score each application
zip_file_path = "/content/Webinar_resumes.zip" # For example give the path to resume_data.zip

def extract_and_rename(zip_file_path, extract_path="extracted_files"):
    """
    Extract files from a zip archive to a specified directory.
    Rename directories containing spaces to use underscores instead.

    Args:
    - zip_file_path (str): The path to the zip file to be extracted.
    - extract_path (str, optional): The path where the zip file content should be extracted to.
                                    Defaults to "extracted_files".

    Returns:
    - str: Path to the resume or directory.
    """
    # Check if extract_path exists, if not, create it
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)

    # If extract_path is not empty, skip extraction
    if not os.listdir(extract_path):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

    resume_path = extract_path
    for item in os.listdir(extract_path):
        item_path = os.path.join(extract_path, item)

        # Check if the current item is a directory and if it has spaces in its name
        if os.path.isdir(item_path) and ' ' in item:
            new_name = item.replace(' ', '_')
            new_path = os.path.join(extract_path, new_name)

            # If the new directory name doesn't already exist, create it
            if not os.path.exists(new_path):
                os.makedirs(new_path)

            # Copying contents from the old directory to the new one
            for sub_item in os.listdir(item_path):
                shutil.copy2(os.path.join(item_path, sub_item), new_path)

            # Removing the old directory
            shutil.rmtree(item_path)
            resume_path = new_path
        else:
            resume_path = item_path

    return resume_path
resume_path = extract_and_rename(zip_file_path)


def process_applications(json_data, must_have_skills, yoe):

    for application in json_data:
        # print(application, "application")
        app_data = {}
        if 'resume_path' in application and 'email_id' in application:
            resume_text = read_document(os.path.join(resume_path, application['resume_path']))
            resume_text, _, _ = check_and_trim(resume_text)
            resume_summary = application['resume_summary']
            application["name_of_candidate"] = resume_summary["name_of_candidate"]
            application["years_of_experience"] = resume_summary["years_of_experience"]
            skill_score, len_must_have, store_js, skill_data = calculate_score(resume_summary["technical_skills"], resume_text, must_have_skills)
            application["len_must_have"] = len_must_have
            application["skills"] = store_js
            yoe = extract_years(str(yoe))  # Define appropriately
            print(application, "resume_summary[name_of_candidate]")
            application = years_of_exp(application, yoe, skill_score)
            name = application["name_of_candidate"]
            score = application["score"]


            # print("resume summary", resume_summary)

    return json_data


applications = process_applications(json_data, must_have_skills, yoe)
application = rank_applications(applications)
for app in application:
    name = app["name_of_candidate"]
    yoe = app["years_of_experience"]
    skill_score = app["skill_score"]
    store_js = app["skills"]
    final_score = app["mean"]  # Assuming mean is the final score

    print(f"[Final Ranking] Candidate Name: {name}")
    print(f"[Final Ranking] Years of Experience: {yoe}")
    print(f"[Final Ranking] Skill Score: {skill_score}")
    print(f"[Final Ranking] Skills: {store_js}")
    print(f"[Final Ranking] Final Score: {final_score}")
    print("-" * 40)  # Separator

{'current_ctc': 125503.5, 'expected_ctc': 138444.2, 'willing_to_relocate': 'yes', 'current_location': 'San Francisco', 'notice_period': '34 days', 'email_id': 'Julian-Barcenas-Fake-Resume-64fbe45abfd46@example.com', 'resume_path': 'Julian-Barcenas-Fake-Resume-64fbe45abfd46.pdf', 'resume_summary': {'name_of_candidate': 'Julian Barcenas', 'mobile_number': '(604) 123-4567', 'email_id': 'julian.barcenas@email.com', 'years_of_experience': 5, 'education': 'Master’s in Computer Science (Specialization in Machine Learning)', 'university': 'University of British Columbia', 'linkedin_profile': 'linkedin.com/in/julianbarcenas', 'technical_skills': ['Python', 'R', 'Machine Learning Algorithms', 'TensorFlow', 'PyTorch', 'scikit-learn', 'Data Preprocessing Tools', 'Matplotlib', 'Seaborn', 'AWS'], 'years_of_jobs': ['2018-2023'], 'year_in_current_position': 2, 'Present_Organization': 'TechFlow Corp', 'summary': "Julian Barcenas is an AI/ML Specialist with 5 years of experience in the development and d

# Output

```
[Final Ranking] Candidate Name: Yogessh J Bhati
[Final Ranking] Years of Experience: 5
[Final Ranking] Skill Score: 5.0
[Final Ranking] Skills: {'PyTorch': 5, 'Computer Vision': 5, 'Keras': 5, 'TensorFlow': 5}
[Final Ranking] Final Score: 1.1601474805833616
----------------------------------------
[Final Ranking] Candidate Name: PANKAJ KUMAR GOYAL
[Final Ranking] Years of Experience: 3
[Final Ranking] Skill Score: 1.8
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 5, 'Keras': 2, 'TensorFlow': 2}
[Final Ranking] Final Score: 0.5599999999999999
----------------------------------------
[Final Ranking] Candidate Name: Soso Sukhitashvili
[Final Ranking] Years of Experience: 5
[Final Ranking] Skill Score: 1.5
[Final Ranking] Skills: {'PyTorch': 5, 'Computer Vision': 5, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.46014748058336163
----------------------------------------
[Final Ranking] Candidate Name: SAI KIRAN
[Final Ranking] Years of Experience: 2
[Final Ranking] Skill Score: 1.05
[Final Ranking] Skills: {'PyTorch': 5, 'Computer Vision': 2, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.388967863362874
----------------------------------------
[Final Ranking] Candidate Name: Naren Sadhwani
[Final Ranking] Years of Experience: 9
[Final Ranking] Skill Score: 1.5
[Final Ranking] Skills: {'PyTorch': 5, 'Computer Vision': 5, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.32706705664732255
----------------------------------------
[Final Ranking] Candidate Name: Ayşin Sancı
[Final Ranking] Years of Experience: 14
[Final Ranking] Skill Score: 1.5
[Final Ranking] Skills: {'PyTorch': 5, 'Computer Vision': 5, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.30024077199896565
----------------------------------------
[Final Ranking] Candidate Name: Joseph Adeola
[Final Ranking] Years of Experience: 4
[Final Ranking] Skill Score: 0.5
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 5, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.28919189378135307
----------------------------------------
[Final Ranking] Candidate Name: AHMED YASSIN
[Final Ranking] Years of Experience: 1
[Final Ranking] Skill Score: 0.6
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 2, 'Keras': 0, 'TensorFlow': 2}
[Final Ranking] Final Score: 0.24823607768599093
----------------------------------------
[Final Ranking] Candidate Name: Lucky Dube
[Final Ranking] Years of Experience: 2
[Final Ranking] Skill Score: 0.2
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 2, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.21896786336287394
----------------------------------------
[Final Ranking] Candidate Name: Abhilash Babu
[Final Ranking] Years of Experience: 18
[Final Ranking] Skill Score: 1.05
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 5, 'Keras': 0, 'TensorFlow': 2}
[Final Ranking] Final Score: 0.21000074533063443
----------------------------------------
[Final Ranking] Candidate Name: Julian Barcenas
[Final Ranking] Years of Experience: 5
[Final Ranking] Skill Score: 0.2
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 0, 'Keras': 0, 'TensorFlow': 2}
[Final Ranking] Final Score: 0.20014748058336163
----------------------------------------
[Final Ranking] Candidate Name: Jose Jesus Cabrera Pantoja
[Final Ranking] Years of Experience: 5
[Final Ranking] Skill Score: 0.2
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 2, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.20014748058336163
----------------------------------------
[Final Ranking] Candidate Name: Tahsin Samia
[Final Ranking] Years of Experience: 4
[Final Ranking] Skill Score: 0.0
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 0, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.1891918937813531
----------------------------------------
[Final Ranking] Candidate Name: Paula Ramos
[Final Ranking] Years of Experience: 9
[Final Ranking] Skill Score: 0.5
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 5, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.12706705664732254
----------------------------------------
[Final Ranking] Candidate Name: Derrick I.C. VAN FRAUSUM
[Final Ranking] Years of Experience: 9
[Final Ranking] Skill Score: 0.0
[Final Ranking] Skills: {'PyTorch': 0, 'Computer Vision': 0, 'Keras': 0, 'TensorFlow': 0}
[Final Ranking] Final Score: 0.027067056647322542
----------------------------------------
```

